# Neural Network with SPU

>  Please read lab [Logistic Regression On SPU](./lr_with_spu.ipynb) first if you have not。

In lab [Logistic Regression On SPU](./lr_with_spu.ipynb), we have showed how to use SecretFlow/SPU to convert a plaintext JAX training program to a secure MPC training program.

In this lab, the idea is quite similar but this time we will work with a Neural Network model.

We are going to use the same dataset and all the settings as lab [Logistic Regression On SPU](./lr_with_spu.ipynb).

And first, let's work out the plaintext model.

>The following codes are demos only. It's **NOT for production** due to system security concerns, please **DO NOT** use it directly in production.

> This tutorial needs more resources than 8c16g, which is the minimum requirement of SecretFlow.

## Train a model with JAX/FLAX

### Load the Dataset

The below is just copied from lab [Logistic Regression On SPU](./lr_with_spu.ipynb). I'm not going to explain again.

In [ ]:
import sys

# use new flax version for compatibility
# fix numpy version to prevent install numpy2 and break other dependencies
!{sys.executable} -m pip install flax==0.8.4 numpy==1.23.5

In [2]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import jax



def breast_cancer(party_id=None, train: bool = True) -> (np.ndarray, np.ndarray):
    x, y = load_breast_cancer(return_X_y=True)
    x = (x - np.min(x)) / (np.max(x) - np.min(x))
    x_train, x_test, y_train, y_test = train_test_split(
        x, y, test_size=0.2, random_state=42
    )

    if train:
        if party_id:
            if party_id == 1:
                return x_train[:, :15], _
            else:
                return x_train[:, 15:], y_train
        else:
            return x_train, y_train
    else:
        return x_test, y_test

### Define the Model


We are going to use a 4-layer [MLP](https://en.wikipedia.org/wiki/Multilayer_perceptron) model with a [ReLU](https://en.wikipedia.org/wiki/Rectifier_(neural_networks)) activation function here.

In [3]:
from typing import Sequence
import flax.linen as nn


FEATURES = [30, 15, 8, 1]


class MLP(nn.Module):
    features: Sequence[int]

    @nn.compact
    def __call__(self, x):
        for feat in self.features[:-1]:
            x = nn.relu(nn.Dense(feat)(x))
        x = nn.Dense(self.features[-1])(x)
        return x

Then we define the training method here.

In [4]:
import jax.numpy as jnp


def predict(params, x):
    # TODO(junfeng): investigate why need to have a duplicated definition in notebook,
    # which is not the case in a normal python program.
    from typing import Sequence
    import flax.linen as nn

    FEATURES = [30, 15, 8, 1]

    class MLP(nn.Module):
        features: Sequence[int]

        @nn.compact
        def __call__(self, x):
            for feat in self.features[:-1]:
                x = nn.relu(nn.Dense(feat)(x))
            x = nn.Dense(self.features[-1])(x)
            return x

    return MLP(FEATURES).apply(params, x)


def loss_func(params, x, y):
    pred = predict(params, x)

    def mse(y, pred):
        def squared_error(y, y_pred):
            return jnp.multiply(y - y_pred, y - y_pred) / 2.0

        return jnp.mean(squared_error(y, pred))

    return mse(y, pred)


def train_auto_grad(x1, x2, y, params, n_batch=10, n_epochs=10, step_size=0.01):
    x = jnp.concatenate((x1, x2), axis=1)
    xs = jnp.array_split(x, len(x) / n_batch, axis=0)
    ys = jnp.array_split(y, len(y) / n_batch, axis=0)

    def body_fun(_, loop_carry):
        params = loop_carry
        for x, y in zip(xs, ys):
            _, grads = jax.value_and_grad(loss_func)(params, x, y)
            params = jax.tree_util.tree_map(
                lambda p, g: p - step_size * g, params, grads
            )
        return params

    params = jax.lax.fori_loop(0, n_epochs, body_fun, params)
    return params


def model_init(n_batch=10):
    model = MLP(FEATURES)
    return model.init(jax.random.PRNGKey(1), jnp.ones((n_batch, FEATURES[0])))

### Validate the Model

We use AUC as the validation metric.

In [5]:
from sklearn.metrics import roc_auc_score


def validate_model(params, X_test, y_test):
    y_pred = predict(params, X_test)
    return roc_auc_score(y_test, y_pred)

### BUILD Together

Let's put everything together and train a plaintext NN model!

In [6]:
# Load the data
x1, _ = breast_cancer(party_id=1, train=True)
x2, y = breast_cancer(party_id=2, train=True)


# Hyperparameter
n_batch = 10
n_epochs = 10
step_size = 0.01


# Train the model
init_params = model_init(n_batch)
params = train_auto_grad(x1, x2, y, init_params, n_batch, n_epochs, step_size)

# Test the model
X_test, y_test = breast_cancer(train=False)
auc = validate_model(params, X_test, y_test)
print(f'auc={auc}')

auc=0.9927939731411726


Must keep the number of AUC in mind, we are going to repeat the training with SPU. Let's do that magic!


## Train a Model with SPU

In [ ]:
import secretflow as sf

# Check the version of your SecretFlow
print('The version of SecretFlow: {}'.format(sf.__version__))

# In case you have a running secretflow runtime already.
# in new version of sf.shutdown() will return Error if no runtime is found.
try:
    sf.shutdown()
except:
    pass

sf.init(['alice', 'bob'], address='local')

alice, bob = sf.PYU('alice'), sf.PYU('bob')
spu = sf.SPU(sf.utils.testing.cluster_def(['alice', 'bob']))

x1, _ = alice(breast_cancer)(party_id=1, train=True)
x2, y = bob(breast_cancer)(party_id=2, train=True)
init_params = model_init(n_batch)


device = spu
x1_, x2_, y_ = x1.to(device), x2.to(device), y.to(device)
init_params_ = sf.to(alice, init_params).to(device)

params_spu = spu(train_auto_grad, static_argnames=['n_batch', 'n_epochs', 'step_size'])(
    x1_, x2_, y_, init_params_, n_batch=n_batch, n_epochs=n_epochs, step_size=step_size
)

The version of SecretFlow: 1.11.0b1


/home/zoupeicheng.zpc/miniconda3/envs/sf/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-25 16:22:04,154	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-03-25 16:22:04,276	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/home/zoupeicheng.zpc/miniconda3/envs/sf/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
2025-03-25 16:22:05,905	INFO worker.py:1724 -- Started a local Ray instance.


(pyu_fn pid=58922) [2025-03-25 16:22:08.052] [warning] [openssl_factory.cc:83] Yacl has been configured to use Yacl's entropy source, but unable to find one. Fallback to use openssl's default entropy srouce
(pyu_fn pid=58922) [2025-03-25 16:22:08.053] [warning] [openssl_factory.cc:83] Yacl has been configured to use Yacl's entropy source, but unable to find one. Fallback to use openssl's default entropy srouce


(pyu_fn pid=58923) 2025-03-25 16:22:08,017,017 INFO [xla_bridge.py:backends:863] Unable to initialize backend 'cuda': 
(pyu_fn pid=58923) 2025-03-25 16:22:08,017,017 INFO [xla_bridge.py:backends:863] Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
(pyu_fn pid=58923) 2025-03-25 16:22:08,017,017 INFO [xla_bridge.py:backends:863] Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory


Let's check params from SPU program.

In [8]:
params_spu = spu(train_auto_grad)(x1_, x2_, y_, init_params)
params = sf.reveal(params_spu)
print(params)

{'params': {'Dense_0': {'bias': array([ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, -8.4447712e-03,
        4.7277778e-02,  3.7619472e-04,  0.0000000e+00,  4.5648813e-03,
        0.0000000e+00, -3.4031734e-02, -8.4131360e-03,  0.0000000e+00,
        0.0000000e+00,  5.6682736e-02, -4.8433840e-03,  0.0000000e+00,
        3.5732210e-02,  6.3549578e-03,  2.9711574e-03,  3.2665402e-02,
        0.0000000e+00, -2.1323502e-02, -7.8181922e-03,  0.0000000e+00,
        2.8501630e-02,  0.0000000e+00, -3.0902624e-03,  3.8683414e-05,
        1.4437497e-02,  2.0847648e-02], dtype=float32), 'kernel': array([[-0.14871399, -0.23531966, -0.1493772 , -0.01558919, -0.13323164,
         0.19175903, -0.03680335, -0.03745084, -0.14176767,  0.03231028,
         0.12652716, -0.40251398, -0.16895528,  0.21399274, -0.13845327,
         0.10585146, -0.11602777,  0.38624388,  0.05965905,  0.06317489,
         0.07793002, -0.01319648, -0.28804976, -0.09602834,  0.111113  ,
        -0.08544238,  0.07546462, -0.041

Lastly, let's validate the model.

In [9]:
X_test, y_test = breast_cancer(train=False)
auc = validate_model(params, X_test, y_test)
print(f'auc={auc}')

auc=0.9927939731411726


This is the end of the lab.